# Tutorial for the priority queueing network calculation (with priority discipline at nodes)
### Compare results with numerical calculations using decomposition method.

In [ ]:
# import libs
import numpy as np

from most_queue.io.tables import print_sojourn_multiclass
from most_queue.random.distributions import H2Distribution
from most_queue.sim.networks.priority_network import PriorityNetwork
from most_queue.theory.networks.open_network_prty import OpenNetworkCalcPriorities

### Set network parameters

In [2]:
k_num = 3  # Number of job classes
n_num = 5  # Number of network nodes

n = [3, 2, 3, 4, 3]  # Distribution of channels (servers) at each node
R = []  # List of transition probability matrices for each class
b = []  # List of service time moments for each class and node
for i in range(k_num):
    R.append(
        np.matrix(
            [
                [1, 0, 0, 0, 0, 0],
                [0, 0.4, 0.6, 0, 0, 0],
                [0, 0, 0, 0.6, 0.4, 0],
                [0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 1],
            ]
        )
    )
L = [0.1, 0.3, 0.4]  # Arrival rates for each class
nodes_prty = []  # Priority distribution at each node: [node][class]

jobs_num = 200000  # Number of jobs simulated

utilization_for_node = 0.7  # Target utilization per node in the network
serve_cv = 1.2  # Coefficient of variation for service times (higher = more variability)

### Set servers parameters and nodes 
Use H<sub>2</sub> distribution for serving

In [3]:
serv_params = []

h2_params = []
for m in range(n_num):

    b1 = utilization_for_node * n[m] / sum(L)
    h2_params.append(H2Distribution.get_params_by_mean_and_cv(b1, serve_cv))

    serv_params.append([])
    for i in range(k_num):
        serv_params[m].append({'type': 'H', 'params': h2_params[m]})

for k in range(k_num):
    b.append([])
    for m in range(n_num):
        b[k].append(H2Distribution.calc_theory_moments(h2_params[m], 4))

### Set nodes priorities order nodes_prty
It looks like [m][x1, x2 .. x_k], where
- m - node number, 
- xi - priority for i-th class, 
- k - number of classes

### For example: 
- [0][0,1,2] - for the first node, a direct order of priorities is set,
- [2][0,2,1] - for the third node, such an order of priorities is set: for the first class - the oldest (0), for the second - the youngest (2), for the third - intermediate (1)



In [4]:
for m in range(n_num):
    nodes_prty.append([])
    for j in range(k_num):
        if m % 2 == 0:
            # Even-numbered nodes: priorities are assigned in ascending order (0, 1, 2, ...)
            nodes_prty[m].append(j)
        else:
            # Odd-numbered nodes: priorities are assigned in descending order (..., 2, 1, 0)
            nodes_prty[m].append(k_num - j - 1)

### Set priority discipline for each node
The service discipline in the node can be any from the list and is not necessarily the same for each node. 
In this case, we will set them to be the same and equal to NP.

Variants of priorities:
- No  - no priorities, FIFO
- PR  - preemptive resume, with resuming interrupted request
- RS  - preemptive repeat with resampling, re-sampling duration for new service
- RW  - preemptive repeat without resampling, repeating service with previous duration
- NP  - non preemptive, relative priority

In [5]:
prty = ['NP'] * n_num

### Run simulation

In [6]:
# Create simulation of priority network:
qn = PriorityNetwork(k_num)

qn.set_sources(L=L, R=R)
qn.set_nodes(serv_params=serv_params, n=n, prty=prty, nodes_prty=nodes_prty)

#  Run simulation of priority network:
sim_results = qn.run(jobs_num)

print(sim_results)

  0%|          | 0/100 [00:00<?, ?it/s]

Job served: 2000/200000:   1%|          | 1/100 [00:00<00:07, 13.08it/s]2000/200000:   2%|▏         | 2/100 [00:00<00:07, 13.20it/s]4000/200000:   2%|▏         | 2/100 [00:00<00:07, 13.20it/s]6000/200000:   3%|▎         | 3/100 [00:00<00:07, 13.20it/s]6000/200000:   4%|▍         | 4/100 [00:00<00:07, 12.94it/s]8000/200000:   4%|▍         | 4/100 [00:00<00:07, 12.94it/s]10000/200000:   5%|▌         | 5/100 [00:00<00:07, 12.94it/s]10000/200000:   6%|▌         | 6/100 [00:00<00:07, 12.94it/s]12000/200000:   6%|▌         | 6/100 [00:00<00:07, 12.94it/s]14000/200000:   7%|▋         | 7/100 [00:00<00:07, 12.94it/s]14000/200000:   8%|▊         | 8/100 [00:00<00:07, 11.88it/s]16000/200000:   8%|▊         | 8/100 [00:00<00:07, 11.88it/s]18000/200000:   9%|▉         | 9/100 [00:00<00:07, 11.88it/s]18000/200000:  10%|█         | 10/100 [00:00<00:07, 11.87it/s]20000/200000:  10%|█         | 10/100 [00:00<00:07, 11.87it/s]22000/200000:  11%|█         | 11/100 [00:00<00:07, 11.87it/s]22000/200000:  

NetworkResultsPriority(v=[[9.578612503791279, 126.81078171799791, 2152.179031667527], [10.235753294847994, 143.6323346332888, 2564.593882649388], [14.217362659363731, 298.20502824946146, 8531.95733300571]], intensities=None, loads=None, duration=7.969088693, arrived=[25223, 74849, 99955], served=[25220, 74847, 99933])


### Run calculation

In [7]:
#  Get raw moments of soujorney time from calculation:
net_calc = OpenNetworkCalcPriorities()
net_calc.set_sources(R=R, L=L)
net_calc.set_nodes(n=n, b=b, prty=prty, nodes_prty=nodes_prty)
calc_results = net_calc.run()

print(calc_results)

NetworkResultsPriority(v=[[np.float64(9.654871278928685), np.float64(129.10772229091094), np.float64(2205.6352300126036)], [np.float64(10.312899887630065), np.float64(145.75383893156157), np.float64(2612.818739144984)], [np.float64(13.93328447012602), np.float64(279.9923920493441), np.float64(7344.284180810235)]], intensities=[array([0.1  , 0.04 , 0.06 , 0.024, 0.1  ]), array([0.3  , 0.12 , 0.18 , 0.072, 0.3  ]), array([0.4  , 0.16 , 0.24 , 0.096, 0.4  ])], loads=[np.float64(0.6999999999999998), np.float64(0.27999999999999997), np.float64(0.41999999999999993), np.float64(0.16800000003237106), np.float64(0.6999999999999998)], duration=0.5595077150000005, arrived=0, served=0)


### Print results for NP priority

In [ ]:
print_sojourn_multiclass(sim_results.v, calc_results.v)

       raw moments of soujorn time in the system        
------------------------------------------------------------
           |               Number of moment                |
    Cls    | --------------------------------------------- |
           |       1       |       2       |       3       |
------------------------------------------------------------
     | Sim |     9.58      |      127      |   2.15e+03    |
  1  |------------------------------------------------------
     | Num |     9.65      |      129      |   2.21e+03    |
------------------------------------------------------------
     | Sim |     10.2      |      144      |   2.56e+03    |
  2  |------------------------------------------------------
     | Num |     10.3      |      146      |   2.61e+03    |
------------------------------------------------------------
     | Sim |     14.2      |      298      |   8.53e+03    |
  3  |------------------------------------------------------
     | Num |     13.9      |

### Repeat all for PR priority

In [ ]:
prty = ['PR'] * n_num  # Absolute priority at each node
qn = PriorityNetwork(k_num)
qn.set_sources(L=L, R=R)
qn.set_nodes(serv_params=serv_params, n=n, prty=prty, nodes_prty=nodes_prty)

sim_results = qn.run(jobs_num)

net_calc = OpenNetworkCalcPriorities()
net_calc.set_sources(R=R, L=L)
net_calc.set_nodes(n=n, b=b, prty=prty, nodes_prty=nodes_prty)
calc_results = net_calc.run()

print_sojourn_multiclass(sim_results.v, calc_results.v)

Job served:    | 0/100 [00:00<?, ?it/s]2000/200000:   1%|          | 1/100 [00:00<00:06, 15.15it/s]2000/200000:   2%|▏         | 2/100 [00:00<00:06, 14.15it/s]4000/200000:   2%|▏         | 2/100 [00:00<00:06, 14.15it/s]6000/200000:   3%|▎         | 3/100 [00:00<00:06, 14.15it/s]6000/200000:   4%|▍         | 4/100 [00:00<00:07, 13.08it/s]8000/200000:   4%|▍         | 4/100 [00:00<00:07, 13.08it/s]10000/200000:   5%|▌         | 5/100 [00:00<00:07, 13.08it/s]10000/200000:   6%|▌         | 6/100 [00:00<00:07, 12.90it/s]12000/200000:   6%|▌         | 6/100 [00:00<00:07, 12.90it/s]14000/200000:   7%|▋         | 7/100 [00:00<00:07, 12.90it/s]14000/200000:   8%|▊         | 8/100 [00:00<00:07, 13.08it/s]16000/200000:   8%|▊         | 8/100 [00:00<00:07, 13.08it/s]18000/200000:   9%|▉         | 9/100 [00:00<00:06, 13.08it/s]18000/200000:  10%|█         | 10/100 [00:00<00:06, 13.03it/s]20000/200000:  10%|█         | 10/100 [00:00<00:06, 13.03it/s]22000/200000:  11%|█         | 11/100 [00:00<00:06

       raw moments of soujorn time in the system        
------------------------------------------------------------
           |               Number of moment                |
    Cls    | --------------------------------------------- |
           |       1       |       2       |       3       |
------------------------------------------------------------
     | Sim |     8.58      |      108      |   1.82e+03    |
  1  |------------------------------------------------------
     | Num |     8.54      |      107      |   1.75e+03    |
------------------------------------------------------------
     | Sim |     9.07      |      120      |    2.1e+03    |
  2  |------------------------------------------------------
     | Num |     10.1      |      140      |   2.51e+03    |
------------------------------------------------------------
     | Sim |     14.8      |      347      |    1.2e+04    |
  3  |------------------------------------------------------
     | Num |     15.9      |